<a href="https://colab.research.google.com/github/nicolaiberk/Apart/blob/master/BaselineBERT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

# all code is based on https://huggingface.co/transformers/custom_datasets.html#fine-tuning-with-native-pytorch-tensorflow

In [ ]:
# define model
from transformers import BertForSequenceClassification
checkpoint_model = 'distilbert-base-german-cased'
model = BertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

In [14]:
# define optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [15]:
# define tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(checkpoint_model)

In [16]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/germanyPPRs.csv', engine="python")
df = df.reset_index()
df = df.dropna()
df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [ ]:
# test data: IMDP from Huggingface tutorial

!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [18]:
# train/test split
trainset = df[df.date < pd.to_datetime('2018-01-01', format='%Y-%m-%d')]
testset = df[df.date >= pd.to_datetime('2018-01-01', format='%Y-%m-%d')]

In [19]:
# validation split
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(trainset['rawtext'], trainset['label'], test_size=.2) # german PPRS
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2) # IMDB

In [20]:
# tokenize
train_encodings = tokenizer([x for x in train_texts], truncation=True, padding=True, max_length=512)
val_encodings = tokenizer([x for x in val_texts], truncation=True, padding=True, max_length=512)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512) #IMDB
test_encodings = tokenizer([x for x in testset['rawtext']], truncation=True, padding=True, max_length=512) # PPRs
test_labels = [x for x in testset['label']] # PPRs

In [21]:
# turn labels and encodings in dataset object
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [22]:
# Try torch alternatively

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained(checkpoint_model)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

KeyError: ignored